# Getting Started

## Introduction
ReSyPE (pronounced *recipe*) is a Python library built for both practitioners and researchers that wish to quickly develop and deploy ML-based recommender systems.

The library provides an end-to-end pipeline that includes:

1. Loading transaction, user feature, and item feature datasets
2. Interchangable methods for user and item clustering
3. Modular framework for machine learning models
4. Iterative and decomposition-based techniques


## Installation

`pip install resype`


## Load the Data

In [26]:
import pandas as pd
import numpy as np
from resype.resype import Resype

%load_ext autoreload
%autoreload 2 

In [27]:
# load transaction list
transaction_list = pd.read_csv("sample_data/ratings.csv")[['userId', 'movieId', 'rating']]
transaction_list = transaction_list.sample(20)
transaction_list.columns = ["user_id", 'item_id', 'rating']

## Preprocess

In [36]:
re = Resype(transaction_list)
re.transaction_list.head(3)

,user_id,item_id,rating
44904,298,6297,2.0
18858,122,778,5.0
83217,527,1270,4.0


In [37]:
# construct utlity matrix
re.construct_utility_matrix()
re.utility_matrix.head(3)

item_id,552,778,956,1046,1193,1270,1380,1438,1498,1625,1752,1962,2947,3052,4024,6182,6297,6370,6755,61024
user_id,,,,,,,,,,,,,,,,,,,,
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN
58,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


## Train on Unclustered Matrix

### Iterative Approach

In [24]:
# import sklearn Model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [38]:
# fit and predict
re.fit(model, method='iterative')
re.utility_matrix_preds.head(3)

item_id,552,778,956,1046,1193,1270,1380,1438,1498,1625,1752,1962,2947,3052,4024,6182,6297,6370,6755,61024
user_id,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)
re.df_rec

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,0,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
1,1,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
2,2,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0


### SVD Approach

In [40]:
# fit and predict
model = RandomForestRegressor()
re.fit(model, method='svd')
re.utility_matrix_preds.head(3)

item_id,552,778,956,1046,1193,1270,1380,1438,1498,1625,1752,1962,2947,3052,4024,6182,6297,6370,6755,61024
user_id,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)
re.df_rec

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,0,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
1,1,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
2,2,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0


## Train on Clustered matrix

In [44]:
from sklearn.cluster import KMeans
km_users = KMeans(n_clusters=10)
km_items = KMeans(n_clusters=10)

user_model, user_cluster_map, util_matrix_w_users = re.cluster_users(km_users)
item_model, item_cluster_map, util_matrix_w_items = re.cluster_items(km_items)

In [45]:
re.cluster_assignment()
re.utility_matrix_agg(u_agg="mean", i_agg="mean")
re.utility_matrix.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,0.0,0.000000,0.0,0.0,0.000000,5.0,0.0,0.0,0.0,0.0
1,0.0,0.219008,0.0,0.0,0.363636,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,5.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [46]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor()
re.fit(model_object=model1, method='iterative', n_synth_data=5, p=0.1)
re.utility_matrix_preds.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,-0.511111,-0.511111,-0.511111,-0.511111,-0.511111,4.488889,-0.511111,-0.511111,-0.511111,0.073475
1,-0.057606,0.139845,-0.069396,-0.035502,0.306031,-0.057606,-0.057606,-0.057606,-0.057606,-0.057606
2,-0.346791,-0.569760,-0.592857,4.407143,-0.505545,0.320191,-0.592857,-0.592857,-0.390167,-0.013302


In [47]:
# predict top item clusters per user
user_list = [0, 1, 2] # index
top_n = 5 # top n clusters
re.get_rec(user_list, top_n)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5
0,0,9.0,8.0,7.0,6.0,4.0
1,1,3.0,9.0,8.0,7.0,6.0
2,2,5.0,9.0,0.0,8.0,4.0


In [ ]:
# predict top items per user
